In [1]:
import json
import pandas as pd
import os

In [2]:
# interactive widge을 위한 패키지
from IPython.display import display
from ipywidgets import interactive, widgets

In [3]:
import folium #지도 

In [4]:
pip install plotly==3.10.0

In [5]:
# plotly: 그래프 패키지
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [6]:
from IPython.display import clear_output
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from folium.plugins import HeatMap, HeatMapWithTime, TimestampedGeoJson

In [7]:
import requests

geo_path = 'https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2013/json/skorea_provinces_geo_simple.json'

res=requests.get(geo_path)
res.raise_for_status()

geo_data=res.json()

In [8]:
original_data = pd.read_csv("/content/drive/MyDrive/캐글스터디/KCB데이터/credit_card_data.csv")

In [9]:
original_data.head(80)

,pop_cd,year,month,avg_score,avg_rat,city,sex,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment
0,L011,2016,1,812,2,서울,NaN,10대,55000,1.44,1.13,70000,92630000,87350000,85790000,0,0,0,0,1120000,8210000,78020000,92170000,87260000,0,0
1,L011,2016,2,773,4,서울,NaN,10대,380000,1.71,0.96,170000,31140000,28910000,25850000,190000,510000,220000,1020000,680000,4660000,23300000,8260000,42410000,4000,1000
2,L011,2016,3,771,4,서울,NaN,10대,387000,1.75,0.95,190000,30230000,28180000,25110000,170000,490000,210000,1140000,640000,4370000,22930000,8300000,41710000,3000,1000
3,L011,2016,4,770,4,서울,NaN,10대,412000,1.76,0.92,200000,30430000,27710000,24630000,190000,510000,240000,1340000,510000,4770000,22160000,9460000,41040000,4000,1000
4,L011,2016,5,771,4,서울,NaN,10대,414000,1.75,0.94,170000,31210000,27620000,24390000,210000,490000,340000,1490000,640000,4740000,21960000,9660000,40850000,3000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,L041,2016,4,765,4,대구,NaN,10대,88000,1.77,0.98,200000,15440000,13150000,8160000,130000,1150000,70000,1790000,510000,2940000,9510000,10100000,15270000,3000,1000
76,L041,2016,5,765,4,대구,NaN,10대,92000,1.75,0.95,160000,15530000,13400000,8430000,120000,1250000,270000,1930000,1410000,3180000,8610000,9780000,16190000,3000,1000
77,L041,2016,6,765,4,대구,NaN,10대,92000,1.76,0.97,180000,13090000,12390000,7020000,150000,1280000,240000,1940000,870000,3300000,8000000,9510000,14360000,3000,1000
78,L041,2016,7,765,4,대구,NaN,10대,91000,1.77,0.99,170000,13560000,12820000,7530000,170000,1490000,30000,2020000,890000,3890000,7820000,9460000,15440000,3000,1000


In [10]:
print(original_data.type)

AttributeError: ignored

In [11]:
# 'year'와 'month'를 합쳐 '2016/01' 형태의 값을 가지는 'time' 컬럼을 생성
tmp = original_data['month'].apply(str)
tmp = tmp.apply(lambda x : '0'+x if len(x)==1 else x)
for index in original_data.index: # loc는 슬라이싱 방법
  original_data.loc[index,'time']=str(original_data.loc[index, 'year']) + '/'+str(tmp.loc[index])

In [12]:
original_data.head()

,pop_cd,year,month,avg_score,avg_rat,city,sex,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment,time
0,L011,2016,1,812,2,서울,NaN,10대,55000,1.44,1.13,70000,92630000,87350000,85790000,0,0,0,0,1120000,8210000,78020000,92170000,87260000,0,0,2016/01
1,L011,2016,2,773,4,서울,NaN,10대,380000,1.71,0.96,170000,31140000,28910000,25850000,190000,510000,220000,1020000,680000,4660000,23300000,8260000,42410000,4000,1000,2016/02
2,L011,2016,3,771,4,서울,NaN,10대,387000,1.75,0.95,190000,30230000,28180000,25110000,170000,490000,210000,1140000,640000,4370000,22930000,8300000,41710000,3000,1000,2016/03
3,L011,2016,4,770,4,서울,NaN,10대,412000,1.76,0.92,200000,30430000,27710000,24630000,190000,510000,240000,1340000,510000,4770000,22160000,9460000,41040000,4000,1000,2016/04
4,L011,2016,5,771,4,서울,NaN,10대,414000,1.75,0.94,170000,31210000,27620000,24390000,210000,490000,340000,1490000,640000,4740000,21960000,9660000,40850000,3000,1000,2016/05


In [13]:
# 지역에 매칭되는 코드
province_code = {'제주': '39',
 '경남': '38',
 '경북': '37',
 '전남': '36',
 '전북': '35',
 '충남': '34',
 '충북': '33',
 '강원': '32',
 '경기': '31',
 '울산': '26',
 '대전': '25',
 '광주': '24',
 '인천': '23',
 '대구': '22',
 '부산': '21',
 '서울': '11'}

In [14]:
print(province_code)

{'제주': '39', '경남': '38', '경북': '37', '전남': '36', '전북': '35', '충남': '34', '충북': '33', '강원': '32', '경기': '31', '울산': '26', '대전': '25', '광주': '24', '인천': '23', '대구': '22', '부산': '21', '서울': '11'}


In [15]:
province_rename = {'서울':'서울특별시',
                   '인천':'인천광역시',
                   '대전':'대전광역시',
                   '대구':'대구광역시',
                   '광주':'광주광역시',
                   '부산':'부산광역시',
                   '울산':'울산광역시',
                   '경기':'경기도',
                   '강원':'강원도',
                   '충북':'충청북도',
                   '충남':'충청남도',
                   '전북':'전라북도',
                   '전남':'전라남도',
                   '경북':'경상북도',
                   '경남':'경상남도',
                   '제주':'제주특별자치도'}

In [16]:
val_cols = ['num_opencard', 'num_usecard',
       'monthly_card_spend', 'monthly_lc', 'monthly_loan', 'monthly_bk_loan',
       'monthly_cd_loan', 'monthly_installments_loan',
       'monthly_insurance_loan', 'monthly_sbk_loan', 'loan_commitment',
       'inst_rep_loanb', 'ls_rep_loanb', 'credit_loan', 'mortgage_loan',
       'credit_card_payment', 'credit_card_installments_payment']

In [17]:
# 지역에 매칭되는 'code' 컬럼 생성
# '10대' 형태의 'ages' 컬럼을 int 10 형태로 변경

original_data['code']= original_data['city'].replace(province_code)
original_data['ages'] = original_data.ages.apply(lambda x: int(x[:2]))



In [18]:
# 인구 수에 비례하도록 수치 값을 보정한 raw_num_opencard 컬럼 생성
for col in val_cols:
  original_data['raw_'+col] = (original_data[col]*original_data['population'])

In [19]:
original_data.head()

,pop_cd,year,month,avg_score,avg_rat,city,sex,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment,time,code,raw_num_opencard,raw_num_usecard,raw_monthly_card_spend,raw_monthly_lc,raw_monthly_loan,raw_monthly_bk_loan,raw_monthly_cd_loan,raw_monthly_installments_loan,raw_monthly_insurance_loan,raw_monthly_sbk_loan,raw_loan_commitment,raw_inst_rep_loanb,raw_ls_rep_loanb,raw_credit_loan,raw_mortgage_loan,raw_credit_card_payment,raw_credit_card_installments_payment
0,L011,2016,1,812,2,서울,NaN,10,55000,1.44,1.13,70000,92630000,87350000,85790000,0,0,0,0,1120000,8210000,78020000,92170000,87260000,0,0,2016/01,11,79200.0,62150.0,3850000000,5094650000000,4804250000000,4718450000000,0,0,0,0,61600000000,451550000000,4291100000000,5069350000000,4799300000000,0,0
1,L011,2016,2,773,4,서울,NaN,10,380000,1.71,0.96,170000,31140000,28910000,25850000,190000,510000,220000,1020000,680000,4660000,23300000,8260000,42410000,4000,1000,2016/02,11,649800.0,364800.0,64600000000,11833200000000,10985800000000,9823000000000,72200000000,193800000000,83600000000,387600000000,258400000000,1770800000000,8854000000000,3138800000000,16115800000000,1520000000,380000000
2,L011,2016,3,771,4,서울,NaN,10,387000,1.75,0.95,190000,30230000,28180000,25110000,170000,490000,210000,1140000,640000,4370000,22930000,8300000,41710000,3000,1000,2016/03,11,677250.0,367650.0,73530000000,11699010000000,10905660000000,9717570000000,65790000000,189630000000,81270000000,441180000000,247680000000,1691190000000,8873910000000,3212100000000,16141770000000,1161000000,387000000
3,L011,2016,4,770,4,서울,NaN,10,412000,1.76,0.92,200000,30430000,27710000,24630000,190000,510000,240000,1340000,510000,4770000,22160000,9460000,41040000,4000,1000,2016/04,11,725120.0,379040.0,82400000000,12537160000000,11416520000000,10147560000000,78280000000,210120000000,98880000000,552080000000,210120000000,1965240000000,9129920000000,3897520000000,16908480000000,1648000000,412000000
4,L011,2016,5,771,4,서울,NaN,10,414000,1.75,0.94,170000,31210000,27620000,24390000,210000,490000,340000,1490000,640000,4740000,21960000,9660000,40850000,3000,1000,2016/05,11,724500.0,389160.0,70380000000,12920940000000,11434680000000,10097460000000,86940000000,202860000000,140760000000,616860000000,264960000000,1962360000000,9091440000000,3999240000000,16911900000000,1242000000,414000000


In [20]:
# 지역정보가 있는 데이터는 성별정보가 없고, 그 반대의 경우도 마찬가지이므로
# 데이터의 중복을 피하기 위해 데이터를 두 가지로 나눔.
province_data = original_data[~original_data.city.isnull()]
sex_data = original_data[~original_data.sex.isnull()]   #?!?!?!

In [21]:
sex_data.head(50)

,pop_cd,year,month,avg_score,avg_rat,city,sex,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment,time,code,raw_num_opencard,raw_num_usecard,raw_monthly_card_spend,raw_monthly_lc,raw_monthly_loan,raw_monthly_bk_loan,raw_monthly_cd_loan,raw_monthly_installments_loan,raw_monthly_insurance_loan,raw_monthly_sbk_loan,raw_loan_commitment,raw_inst_rep_loanb,raw_ls_rep_loanb,raw_credit_loan,raw_mortgage_loan,raw_credit_card_payment,raw_credit_card_installments_payment
384,Y10,2016,1,811,2,NaN,남자,10,960000,1.16,1.05,50000,53760000,48700000,46120000,0,40000,0,0,940000,2310000,45440000,16760000,49540000,0,0,2016/01,NaN,1113600.0,1008000.0,48000000000,51609600000000,46752000000000,44275200000000,0,38400000000,0,0,902400000000,2217600000000,43622400000000,16089600000000,47558400000000,0,0
385,Y10,2016,2,767,4,NaN,남자,10,1144000,1.59,0.90,150000,19520000,16360000,12090000,180000,1470000,400000,300000,1160000,5560000,9340000,7830000,19930000,2000,1000,2016/02,NaN,1818960.0,1029600.0,171600000000,22330880000000,18715840000000,13830960000000,205920000000,1681680000000,457600000000,343200000000,1327040000000,6360640000000,10684960000000,8957520000000,22799920000000,2288000000,1144000000
386,Y10,2016,3,765,4,NaN,남자,10,1168000,1.62,0.90,170000,19050000,16400000,11940000,180000,1490000,470000,260000,1100000,5860000,9160000,7750000,20090000,2000,1000,2016/03,NaN,1892160.0,1051200.0,198560000000,22250400000000,19155200000000,13945920000000,210240000000,1740320000000,548960000000,303680000000,1284800000000,6844480000000,10698880000000,9052000000000,23465120000000,2336000000,1168000000
387,Y10,2016,4,764,4,NaN,남자,10,1211000,1.66,0.89,180000,19400000,16880000,12050000,190000,1520000,670000,220000,1070000,6250000,9260000,8310000,20620000,2000,1000,2016/04,NaN,2010260.0,1077790.0,217980000000,23493400000000,20441680000000,14592550000000,230090000000,1840720000000,811370000000,266420000000,1295770000000,7568750000000,11213860000000,10063410000000,24970820000000,2422000000,1211000000
388,Y10,2016,5,765,4,NaN,남자,10,1216000,1.66,0.91,150000,19570000,16380000,12050000,200000,1650000,690000,220000,580000,6360000,9140000,8630000,19660000,2000,1000,2016/05,NaN,2018560.0,1106560.0,182400000000,23797120000000,19918080000000,14652800000000,243200000000,2006400000000,839040000000,267520000000,705280000000,7733760000000,11114240000000,10494080000000,23906560000000,2432000000,1216000000
389,Y10,2016,6,765,4,NaN,남자,10,1209000,1.67,0.94,170000,18920000,16360000,12120000,210000,1510000,590000,260000,610000,6310000,9080000,8650000,19650000,3000,1000,2016/06,NaN,2019030.0,1136460.0,205530000000,22874280000000,19779240000000,14653080000000,253890000000,1825590000000,713310000000,314340000000,737490000000,7628790000000,10977720000000,10457850000000,23756850000000,3627000000,1209000000
390,Y10,2016,7,766,4,NaN,남자,10,1203000,1.67,0.95,160000,17920000,15980000,11790000,210000,1520000,530000,300000,640000,6200000,8790000,8640000,19240000,2000,1000,2016/07,NaN,2009010.0,1142850.0,192480000000,21557760000000,19223940000000,14183370000000,252630000000,1828560000000,637590000000,360900000000,769920000000,7458600000000,10574370000000,10393920000000,23145720000000,2406000000,1203000000
391,Y10,2016,8,766,4,NaN,남자,10,1198000,1.67,0.97,150000,18680000,16630000,12550000,210000,1490000,480000,290000,610000,6560000,9110000,8520000,20330000,3000,1000,2016/08,NaN,2000660.0,1162060.0,179700000000,22378640000000,19922740000000,15034900000000,251580000000,1785020000000,575040000000,347420000000,730780000000,7858880000000,10913780000000,10206960000000,24355340000000,3594000000,1198000000
392,Y10,2016,9,766,4,NaN,남자,10,1192000,1.67,0.98,170000,19320000,17280000,12630000,210000,1540000,530000,280000,600000,6790000,9550000,8540000,21330000,3000,1000,2

In [22]:
province_data.head(50)

,pop_cd,year,month,avg_score,avg_rat,city,sex,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment,time,code,raw_num_opencard,raw_num_usecard,raw_monthly_card_spend,raw_monthly_lc,raw_monthly_loan,raw_monthly_bk_loan,raw_monthly_cd_loan,raw_monthly_installments_loan,raw_monthly_insurance_loan,raw_monthly_sbk_loan,raw_loan_commitment,raw_inst_rep_loanb,raw_ls_rep_loanb,raw_credit_loan,raw_mortgage_loan,raw_credit_card_payment,raw_credit_card_installments_payment
0,L011,2016,1,812,2,서울,NaN,10,55000,1.44,1.13,70000,92630000,87350000,85790000,0,0,0,0,1120000,8210000,78020000,92170000,87260000,0,0,2016/01,11,79200.0,62150.0,3850000000,5094650000000,4804250000000,4718450000000,0,0,0,0,61600000000,451550000000,4291100000000,5069350000000,4799300000000,0,0
1,L011,2016,2,773,4,서울,NaN,10,380000,1.71,0.96,170000,31140000,28910000,25850000,190000,510000,220000,1020000,680000,4660000,23300000,8260000,42410000,4000,1000,2016/02,11,649800.0,364800.0,64600000000,11833200000000,10985800000000,9823000000000,72200000000,193800000000,83600000000,387600000000,258400000000,1770800000000,8854000000000,3138800000000,16115800000000,1520000000,380000000
2,L011,2016,3,771,4,서울,NaN,10,387000,1.75,0.95,190000,30230000,28180000,25110000,170000,490000,210000,1140000,640000,4370000,22930000,8300000,41710000,3000,1000,2016/03,11,677250.0,367650.0,73530000000,11699010000000,10905660000000,9717570000000,65790000000,189630000000,81270000000,441180000000,247680000000,1691190000000,8873910000000,3212100000000,16141770000000,1161000000,387000000
3,L011,2016,4,770,4,서울,NaN,10,412000,1.76,0.92,200000,30430000,27710000,24630000,190000,510000,240000,1340000,510000,4770000,22160000,9460000,41040000,4000,1000,2016/04,11,725120.0,379040.0,82400000000,12537160000000,11416520000000,10147560000000,78280000000,210120000000,98880000000,552080000000,210120000000,1965240000000,9129920000000,3897520000000,16908480000000,1648000000,412000000
4,L011,2016,5,771,4,서울,NaN,10,414000,1.75,0.94,170000,31210000,27620000,24390000,210000,490000,340000,1490000,640000,4740000,21960000,9660000,40850000,3000,1000,2016/05,11,724500.0,389160.0,70380000000,12920940000000,11434680000000,10097460000000,86940000000,202860000000,140760000000,616860000000,264960000000,1962360000000,9091440000000,3999240000000,16911900000000,1242000000,414000000
5,L011,2016,6,771,4,서울,NaN,10,410000,1.75,0.96,190000,29520000,27020000,23740000,200000,410000,320000,1630000,680000,4730000,21380000,9470000,40130000,4000,2000,2016/06,11,717500.0,393600.0,77900000000,12103200000000,11078200000000,9733400000000,82000000000,168100000000,131200000000,668300000000,278800000000,1939300000000,8765800000000,3882700000000,16453300000000,1640000000,820000000
6,L011,2016,7,772,4,서울,NaN,10,407000,1.75,0.97,180000,28400000,25800000,22140000,220000,520000,320000,1670000,610000,4680000,20230000,8790000,38970000,4000,2000,2016/07,11,712250.0,394790.0,73260000000,11558800000000,10500600000000,9010980000000,89540000000,211640000000,130240000000,679690000000,248270000000,1904760000000,8233610000000,3577530000000,15860790000000,1628000000,814000000
7,L011,2016,8,772,4,서울,NaN,10,403000,1.75,0.98,170000,29170000,26480000,23000000,240000,400000,140000,1720000,830000,4950000,20400000,9660000,40770000,4000,2000,2016/08,11,705250.0,394940.0,68510000000,11755510000000,10671440000000,9269000000000,96720000000,161200000000,56420000000,693160000000,334490000000,1994850000000,8221200000000,3892980000000,16430310000000,1612000000,806000000
8,L011,2016,9,772,4,서울,NaN,10,399000,1.74,0.99,190000,29870000,26910000,23140000,240000,420000,210000,1850000,920000,5240000,20450000,10060000,41590000,5000,2000,2016/09,11,694260.0,395010.0,75810000000,11918130000000,10737090000000,9232860000000,95760000000,167580000000,837

In [23]:
provinces = list(province_code.keys())
ages = [ age for age in range(10,90+1, 10)]

In [24]:
print(provinces)

['제주', '경남', '경북', '전남', '전북', '충남', '충북', '강원', '경기', '울산', '대전', '광주', '인천', '대구', '부산', '서울']


In [25]:
province_data_tmp = province_data.groupby(['city']).sum() #groupby는 그룹별 계산값 출력 , sum이므로 강원에 해당하는 같은 열의 데이터의 합
province_data_tmp.head()


,year,month,avg_score,avg_rat,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment,raw_num_opencard,raw_num_usecard,raw_monthly_card_spend,raw_monthly_lc,raw_monthly_loan,raw_monthly_bk_loan,raw_monthly_cd_loan,raw_monthly_installments_loan,raw_monthly_insurance_loan,raw_monthly_sbk_loan,raw_loan_commitment,raw_inst_rep_loanb,raw_ls_rep_loanb,raw_credit_loan,raw_mortgage_loan,raw_credit_card_payment,raw_credit_card_installments_payment
city,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
강원,435564,1404,171640,708,10800,29946000,475.64,269.57,175770000,11906710000,10241210000,7329850000,122970000,622250000,206120000,170810000,2593580000,2734400000,4717610000,3848710000,12999800000,84383000,30308000,86498570.0,48831030.0,34543610000000,1829943330000000,1571928160000000,1086307580000000,25030570000000,119913050000000,41023010000000,26638110000000,405841450000000,467674050000000,659213810000000,664263280000000,1996601320000000,17135711000000,6100316000000
경기,435564,1404,173791,697,10800,267939000,531.29,297.58,198500000,19605110000,16800210000,14034200000,172650000,521340000,511810000,204340000,2540380000,6085040000,7938390000,4633210000,21241240000,104753000,33161000,906475060.0,502994540.0,358449120000000,23670237910000000,20095112220000000,16054639900000000,316978980000000,959561610000000,815927780000000,302668580000000,2891713600000000,8969255280000000,7801604390000000,6794432220000000,25815855170000000,195612967000000,58458330000000
경남,435564,1404,172615,697,10800,69152000,494.13,273.18,164000000,13628290000,11848140000,9180780000,140480000,542390000,357440000,171440000,2167800000,3446950000,6029820000,4079100000,15289910000,80026000,29813000,217899660.0,120559510.0,78889370000000,5128952620000000,4470988230000000,3361633900000000,68358120000000,255132360000000,174829390000000,64637780000000,789036140000000,1484020900000000,2098116280000000,1592071490000000,5779906590000000,39954515000000,14470024000000
경북,435564,1404,172754,697,10800,53665000,461.30,256.80,150840000,12642820000,10906510000,7860140000,125150000,514580000,240800000,175940000,2466660000,2910660000,5348440000,3712320000,14088300000,75111000,25173000,151281810.0,84623080.0,54533890000000,3581258530000000,3114888300000000,2212163220000000,46130470000000,181327300000000,87860540000000,46676190000000,698177340000000,898922270000000,1451016230000000,1147699000000000,4023414280000000,28031499000000,9367632000000
광주,435564,1404,173895,697,10800,31047000,552.00,296.19,186770000,14046790000,11953680000,8450210000,150300000,475970000,243110000,171530000,2349400000,3661880000,5733260000,4295310000,14904850000,97200000,33506000,111881770.0,59480650.0,38932400000000,2093368750000000,1810000330000000,1267231340000000,31762760000000,102553690000000,48270920000000,29680950000000,371675360000000,630750770000000,763698670000000,743208830000000,2265477380000000,20918703000000,6855986000000


In [26]:
province_data_tmp['city'] = province_data_tmp.index
province_data_tmp['code'] = province_data_tmp['city'].replace(province_code)
province_data_tmp

,year,month,avg_score,avg_rat,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment,raw_num_opencard,raw_num_usecard,raw_monthly_card_spend,raw_monthly_lc,raw_monthly_loan,raw_monthly_bk_loan,raw_monthly_cd_loan,raw_monthly_installments_loan,raw_monthly_insurance_loan,raw_monthly_sbk_loan,raw_loan_commitment,raw_inst_rep_loanb,raw_ls_rep_loanb,raw_credit_loan,raw_mortgage_loan,raw_credit_card_payment,raw_credit_card_installments_payment,city,code
city,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
강원,435564,1404,171640,708,10800,29946000,475.64,269.57,175770000,11906710000,10241210000,7329850000,122970000,622250000,206120000,170810000,2593580000,2734400000,4717610000,3848710000,12999800000,84383000,30308000,86498570.0,48831030.0,34543610000000,1829943330000000,1571928160000000,1086307580000000,25030570000000,119913050000000,41023010000000,26638110000000,405841450000000,467674050000000,659213810000000,664263280000000,1996601320000000,17135711000000,6100316000000,강원,32
경기,435564,1404,173791,697,10800,267939000,531.29,297.58,198500000,19605110000,16800210000,14034200000,172650000,521340000,511810000,204340000,2540380000,6085040000,7938390000,4633210000,21241240000,104753000,33161000,906475060.0,502994540.0,358449120000000,23670237910000000,20095112220000000,16054639900000000,316978980000000,959561610000000,815927780000000,302668580000000,2891713600000000,8969255280000000,7801604390000000,6794432220000000,25815855170000000,195612967000000,58458330000000,경기,31
경남,435564,1404,172615,697,10800,69152000,494.13,273.18,164000000,13628290000,11848140000,9180780000,140480000,542390000,357440000,171440000,2167800000,3446950000,6029820000,4079100000,15289910000,80026000,29813000,217899660.0,120559510.0,78889370000000,5128952620000000,4470988230000000,3361633900000000,68358120000000,255132360000000,174829390000000,64637780000000,789036140000000,1484020900000000,2098116280000000,1592071490000000,5779906590000000,39954515000000,14470024000000,경남,38
경북,435564,1404,172754,697,10800,53665000,461.30,256.80,150840000,12642820000,10906510000,7860140000,125150000,514580000,240800000,175940000,2466660000,2910660000,5348440000,3712320000,14088300000,75111000,25173000,151281810.0,84623080.0,54533890000000,3581258530000000,3114888300000000,2212163220000000,46130470000000,181327300000000,87860540000000,46676190000000,698177340000000,898922270000000,1451016230000000,1147699000000000,4023414280000000,28031499000000,9367632000000,경북,37
광주,435564,1404,173895,697,10800,31047000,552.00,296.19,186770000,14046790000,11953680000,8450210000,150300000,475970000,243110000,171530000,2349400000,3661880000,5733260000,4295310000,14904850000,97200000,33506000,111881770.0,59480650.0,38932400000000,2093368750000000,1810000330000000,1267231340000000,31762760000000,102553690000000,48270920000000,29680950000000,371675360000000,630750770000000,763698670000000,743208830000000,2265477380000000,20918703000000,6855986000000,광주,24
대구,435564,1404,175064,695,10800,52477000,531.23,292.31,166600000,17678600000,13405980000,9932640000,153400000,467900000,456180000,170130000,2396210000,4133980000,6657090000,3768670000,17523570000,87888000,30365000,172200390.0,96801660.0,59664430000000,4200484530000000,3560865110000000,2566158470000000,55126340000000,165202450000000,159677250000000,44667680000000,643043020000000,1320179820000000,1518996820000000,1125140790000000,4826047430000000,32448575000000,10953319000000,대구,22
대전,435564,1404,174630,696,10800,32381000,529.70,295.21,185080000,14668720000,12697770000,9481440000,170080000,468560000,341360000,198350000,2220630000,3684490000,6565990000,4538250000,15637040000,97405000,31391000,108770270.0,60791160.0,40902940000000,2306148170000000,1977168320000000,1441695890000000,37043080000000,103755730000000,6236509

In [27]:
timeline =[]
for year in[ 2016, 2017] :
  for month in range(1, 12+1):
    if month < 10 :
      month = '0'+str(month)
    timeline.append('{0}/{1}'.format(year,month))

In [28]:
print(timeline)

['2016/01', '2016/02', '2016/03', '2016/04', '2016/05', '2016/06', '2016/07', '2016/08', '2016/09', '2016/10', '2016/11', '2016/12', '2017/01', '2017/02', '2017/03', '2017/04', '2017/05', '2017/06', '2017/07', '2017/08', '2017/09', '2017/10', '2017/11', '2017/12']


In [29]:
# 시간 위젯
time_sliders = widgets.SelectionSlider(
    options=['전체']+timeline,
    description='시간',
    disabled=False,
    orientation='horizontal',
    readout=True
)
# 수치 계산 선택 위젯
metric_buttons = widgets.ToggleButtons(
    options=[('합계', 'sum'), ('평균', 1)],
    description='계산 방법:',
)
# 수치 위젯: 어떤 수치를 지도에 표시할 것인지 선택
val_buttons = widgets.Dropdown(
    options=val_cols,
    description='수치:',
)
# 신용점수 선택 위젯
options = range(720, 850+1, 10)
credit_sliders  = widgets.SelectionRangeSlider(
    options=options,
    index=(0,len(options)-1),
    description='신용점수',
    disabled=False
)
# 나이대 선택 위젯
options = range(10, 90+1, 10)
age_sliders = widgets.SelectionRangeSlider(
    options=options,
    index=(0,len(options)-1),
    description='연령대',
    disabled=False,
)

# 위젯을 container로 합침 
container = widgets.VBox([
                widgets.HBox([time_sliders]),
                widgets.HBox([metric_buttons]),
                widgets.HBox([val_buttons]),
                widgets.HBox([credit_sliders]),
                widgets.HBox([age_sliders]),
                ])

In [30]:
def draw_korea_map_val(df, col, metric):
    # 데이터 df를 받아 특정 컬럼 col을 
    # 합계 혹은 평균 metric으로 그룹화하여 지도에 색으로 값을 표시
    
    # 한국 전국 지도

    rome_lat, rome_lng = 36.2002, 127.054
    m=folium.Map(location=[rome_lat,rome_lng], zoom_start=7)

    # 합계 metric일 경우, city와 ages는 합하지 않고 그대로 유지
    if metric=='sum':
        tmp_df = df.groupby(['city'], as_index=False).mean()
        df = df.groupby(['city'], as_index=False).sum()
        df.loc[:, ['ages']] = tmp_df.loc[df.index, ['ages']]
        del tmp_df
    else: # metric=='mean':
        df = df.groupby(['city'], as_index=False).mean()
    
    df['code'] = df['city'].replace(province_code)
    
    # 표시 단위를 적절히 조정하기 위해 MinMaxScaler 적용하여 컬럼 col 값 업데이트
    tmp = df[col].values.reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(0,7))
    scaler.fit(tmp)
    df[col] = scaler.transform(tmp).reshape(-1)

    # geo_data의 key_on 값을 바탕으로 
    # data의 col 값을 색으로 표시한 것을 지도 m에 추가함. 
    folium.Choropleth(
        geo_data=geo_data,
        data = df,
        columns = ['code', col],
        key_on='feature.properties.code',
        fill_color='YlGnBu',
        legend_name=col,
        fill_opacity=0.5,
        line_opacity=0.5,
        #line_weight=2,
    ).add_to(m)

    return m #display(m)


In [31]:
def response(change):
    # 위젯에 변화가 감지되면 실행되는 코드
    
    # 이전 셀 아웃풋은 지움.
    clear_output()  
    
    temp_df = province_data.copy(deep=True) 
    
    # 시간
    if time_sliders.value != '전체':
        temp_df = temp_df[temp_df.time==time_sliders.value]

    # 신용 등급
    min_rat, max_rat = credit_sliders.value
    if min_rat==2 and max_rat==7:
        # not select
        None
    elif min_rat==max_rat:
        temp_df = temp_df[temp_df.avg_score==min_rate]
    else: # min_rat 초과, max_rat 미만
        temp_df = temp_df[(temp_df.avg_score>=max_rat)]

    # 연령
    min_age, max_age = age_sliders.value
    if min_age==24 and max_age==99:
        # not select
        None
    elif min_age==max_age:
        temp_df = temp_df[temp_df.ages==min_age]
    else: # min_age 초과, max_age 미만
        temp_df = temp_df[(temp_df.ages>=max_age)]
    
    
    # 수치, 수치가 0인 row는 제외
    col = val_buttons.value
    temp_df = temp_df[temp_df[col]!=0]
    if col=='medium_resid_rat' or col=='large_resid_rat':
        temp_df.dropna(inplace=True)

    my_map = draw_korea_map_val(temp_df, col, metric_buttons.value)
    
    # 위젯 출력
    display(container)
    
    print ('선택된 그룹은 전체 그룹 중 {0:.1f}% ({1:d}명)를 차지합니다.'.format(len(temp_df)/len(province_data)*100, len(temp_df)))
    # 지도 출력
    display(my_map)

In [28]:
# 위젯 변화를 observe하여 response 함수를 실행
credit_sliders.observe(response, names="value")
val_buttons.observe(response, names="value")
age_sliders.observe(response, names="value")
metric_buttons.observe(response, names="value")
time_sliders.observe(response, names="value")

In [32]:
# 위젯을 표시함. 
# 시간, 계산 방법, 수치, 신용점수, 연령대에 따라 수치가 어떻게 변하는지
# 한 눈에 알아볼 수 있도록 그래프 생성
display(container)
my_map = draw_korea_map_val(province_data, val_buttons.value, metric_buttons.value)
display(my_map)